## --- [Day 13: Shuttle Search](https://adventofcode.com/2020/day/13) ---

In [1]:
# Puzzle inputs
example_input_a = 939
example_input_b = '7,13,x,x,59,x,31,19'

input_a = 1013728
input_b = '23,x,x,x,x,x,x,x,x,x,x,x,x,41,x,x,x,x,x,x,x,x,x,733,x,x,x,x,x,x,x,x,x,x,x,x,13,17,x,x,x,x,19,x,x,x,x,x,x,x,x,x,29,x,449,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,37'

In [2]:
def parse_id_str(id_str):
    '''
    :param id_str: comma-delimited string of integer ID numbers, or 'x' for out-of-service (ignored)
    :return: a list of in-service routes
    '''
    ids = []
    for id in id_str.split(','):
        try:
            ids.append(int(id))
        except ValueError:
            continue
    return ids

def find_next_departure(start_time, ids):
    '''
    Part 1: find the next soonest departure on any route
    :param start_time: earliest possible departure time
    :param ids: IDs of in-service routes
    '''
    elapsed = 0
    while True:
        for id in ids:
            if (start_time + elapsed) % id == 0:
                return (id, elapsed)
        elapsed += 1

In [3]:
# Test against the example data
example_ids = parse_id_str(example_input_b)
assert example_ids == [7,13,59,31,19]
assert find_next_departure(example_input_a, example_ids) == (59,5)

In [4]:
# Part 1 solution
ids = parse_id_str(input_b)
next = find_next_departure(input_a, ids)
print(f'Next departure on bus {next[0]} in {next[1]} minutes; solution is {next[0] * next[1]}')

Next departure on bus 733 in 11 minutes; solution is 8063


### Part 2
What is the earliest timestamp such that all of the listed bus IDs depart at offsets matching their positions in the list?

In [5]:
def parse_id_full(id_str):
    '''
    Same as parse_id_str above, but keep the 'x' values intact
    '''
    ids = []
    for id in id_str.split(','):
        try:
            ids.append(int(id))
        except ValueError:
            ids.append(id)
    return ids

def find_i_max(ids):
    i_max = None
    
    for i in range(len(ids)):
        if isinstance(ids[i], int):
            if i_max is None:
                i_max = i
            elif ids[i] > ids[i_max]:
                i_max = i
    return i_max

def search(id_str):
    '''
    Brute-force (non)solution to part 2
    '''
    ids = parse_id_full(id_str)
    i_max = find_i_max(ids)
    val_max = ids[i_max]
    result = None
    
    # Start below at the first possible occurence, which is val_max - i_max
    elapsed = -i_max
    
    while not result:
        elapsed += val_max
        result = True
        for i in range(len(ids)):
            if ids[i] != 'x':
                result &= (elapsed + i) % ids[i] == 0
    
    return elapsed    

In [6]:
# Test the example
assert search(example_input_b) == 1068781
assert search('17,x,13,19') == 3417
assert search('67,7,59,61') == 754018
assert search('67,7,x,59,61') == 1261476
assert search('1789,37,47,1889') == 1202161486

In [7]:
# Part 2 solution, take 2: brute force is not realistic for the full input, so we have to use some math.
def mod_inv(a, n):
    '''
    Extended Euclidean Algorithm for computing multiplicative inverse of a modulo n
    from here: https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Computing_multiplicative_inverses_in_modular_structures
    '''
    t = 0
    tn = 1
    r = n
    rn = a
    
    while rn != 0:
        quotient = r // rn
        t, tn = tn, t - quotient * tn
        r, rn = rn, r - quotient * rn
    
    if r > 1:
        print(f'No mod inv of {a} mod {n} = {t}')
        return None
    if t < 0:
        t += n

    return t

def solve_system(congruencies):
    '''
    Using Chinese Remainder Theorem as helpfully explained here
    https://www.youtube.com/watch?v=2-tdwLqyaKo
    https://www.youtube.com/watch?v=OB1OcmVSWLc

    Solve a system of congruencies in the format of: x congruent to ai (mod mi)
    i.e., x % mi == ai
    :param congruencies: List of tuples (ai, mi)
    :return: solved value for x
    '''
    # M is the product of all values of m
    M = 1
    for _, mi in congruencies:
        M *= mi
    
    # Calculate Mi, which is the product of all *other* m for each i, i.e., M/Mi
    Mi = []
    # ... and yi, which is the modular inverse of Mi mod mi
    yi = []
    for _, mi in congruencies:
        Mi.append(M//mi)
        yi.append(mod_inv(M//mi, mi))
    
    # Result is (summation of aiMiyi) mod M
    result = 0
    for i in range(len(congruencies)):
        result += congruencies[i][0] * Mi[i] * yi[i]
    
    return result % M

def search_v2(id_str, verbose=False):
    '''
    Part 2 solution again, feasible for larger values
    :param id_str: Input string as described in the puzzle text
    :param verbose: Prints the congruencies for debugging
    '''
    ids = parse_id_full(id_str)
    
    # Build the system of congruencies from the inputs
    congruencies = []
    for i in range(len(ids)):
        if ids[i] != 'x':
            # ai depends on the offset -- elapsed mod id will = 0 when elapsed + i
            ai = (ids[i] - i) % ids[i]
            congruencies.append((ai, ids[i]))
            if verbose:
                print(f'[offset {i}] x is congruent to {ai} (mod {ids[i]})')
            
    return solve_system(congruencies)
    


In [8]:
# Test system of congruencies (not from the puzzle)
# x congruent to 6 (mod 7)
# x congruent to 4 (mod 8)
test_congruencies = [(6,7),(4,8)]
assert solve_system(test_congruencies) == 20

In [9]:
# Test against examples
assert search_v2(example_input_b, True) == 1068781
assert search_v2('17,x,13,19') == 3417
assert search_v2('67,7,59,61') == 754018
assert search_v2('67,x,7,59,61') == 779210
assert search_v2('67,7,x,59,61') == 1261476
assert search_v2('1789,37,47,1889') == 1202161486

[offset 0] x is congruent to 0 (mod 7)
[offset 1] x is congruent to 12 (mod 13)
[offset 4] x is congruent to 55 (mod 59)
[offset 6] x is congruent to 25 (mod 31)
[offset 7] x is congruent to 12 (mod 19)


In [10]:
# Finally, Part 2 solution
search_v2(input_b)

775230782877242